In [1]:
import numpy as np

shell_head = """#!/bin/sh
#SBATCH -J torch
#SBATCH -p xhhgnormal   #修改队列名称，whichpartition查看队列名称
#SBATCH -N 1
#SBATCH -n 16
#SBATCH --gres=gpu:1
set -x
source ~/.bashrc
module load nvidia/cuda/11.6
conda activate RL-torch
"""


In [2]:
list_j = [5, 8, 11, 14, 17, 20, 23, 25]
list_m = [5, 8, 11, 14, 17, 20, 23, 25]

exp="DAN_j_m"

for n_j in list_j:

    commd_list = []
    for n_m in list_m:
        commd = f'python ./test_script/trained_model_ec.py --fea_j_input_dim 12 --fea_m_input_dim 9 --model_source SD2EC0 --data_source SD2EC0 --seed_train 234 --test_model {n_j}x{n_m}EC+ECMK --test_data {n_j}x{n_m}+mix'
        commd_list.append(commd)
    commds = "\n".join(commd_list)

print(commds)
# with open(f"./DANEC_test.sh", "w") as f:
#     s = f'''{shell_head}\n{commds}'''
#     f.write(s)


python ./test_script/trained_model_ec.py --fea_j_input_dim 12 --fea_m_input_dim 9 --model_source SD2EC0 --data_source SD2EC0 --seed_train 234 --test_model 25x5EC+ECMK --test_data 25x5+mix
python ./test_script/trained_model_ec.py --fea_j_input_dim 12 --fea_m_input_dim 9 --model_source SD2EC0 --data_source SD2EC0 --seed_train 234 --test_model 25x8EC+ECMK --test_data 25x8+mix
python ./test_script/trained_model_ec.py --fea_j_input_dim 12 --fea_m_input_dim 9 --model_source SD2EC0 --data_source SD2EC0 --seed_train 234 --test_model 25x11EC+ECMK --test_data 25x11+mix
python ./test_script/trained_model_ec.py --fea_j_input_dim 12 --fea_m_input_dim 9 --model_source SD2EC0 --data_source SD2EC0 --seed_train 234 --test_model 25x14EC+ECMK --test_data 25x14+mix
python ./test_script/trained_model_ec.py --fea_j_input_dim 12 --fea_m_input_dim 9 --model_source SD2EC0 --data_source SD2EC0 --seed_train 234 --test_model 25x17EC+ECMK --test_data 25x17+mix
python ./test_script/trained_model_ec.py --fea_j_input

In [9]:

with open(f"./DAN_j_m/DAN_all.sh", "w") as f:
    s = "\n".join([f"sbatch ./experiment/DAN_j_m/DAN_{n_j}_m.sh" for n_j in range(30, 4, -1)])
    f.write(s)


In [54]:
EXP="exp18"
SUBEXP="exp18-10"
hidden_dim=512


# 本试验特殊参数
# n_j_options=[int(_) for _ in np.linspace(5, 25, 6)]
# n_m_options=n_j_options[::-1]
# op_per_job_options=[10 for _ in n_m_options]

n_j_options=[10, 25, 20, 15]
n_m_options=[5,  5,  10, 15]
op_per_job_options=n_m_options
print(n_j_options, n_m_options, op_per_job_options)

LOG_DIR="./runs/"+EXP+"/"+SUBEXP

num_tasks=len(n_j_options)
seed_train=23333

# multi_task_maml_exp.py 脚本的特定参数
meta_iterations=200+int(SUBEXP.split("-")[-1])
print(meta_iterations)
max_updates_maml=500

# DAN_finetuning.py 脚本的特定参数
max_updates_finetune=11
exp_dim="jxm"

def finetuning_dan(model_list, file, extra_parameters:str=""):
    logdir_dan = LOG_DIR + "/DAN/finetuning"
    commd_list = []
    target_models = []
    for model in model_list:
        for n_j in n_j_options:
            for n_m in n_m_options:
                target_model = f"{n_j}x{n_m}"
                if target_model not in target_models:
                    target_models.append(target_model)
                    commd = f"python ./train/DAN_finetuning.py --logdir {logdir_dan}/{model}/{target_model} --model_suffix free --finetuning_model {model} --max_updates {max_updates_finetune} --n_j {n_j} --n_m {n_m} --exp_dim {exp_dim} {extra_parameters}"
                    print(commd, file=file)
                    commd_list.append(commd)
                else:pass
    return commd_list

def train_maml(file):
    model_suffix = SUBEXP
    logdir_maml=LOG_DIR+ "/maml/train_model"
    commd = f'python ./train/multi_task_maml_exp18.py --logdir {logdir_maml} --model_suffix {model_suffix} --maml_model True \
--meta_iterations {meta_iterations} --num_tasks {num_tasks} \
--hidden_dim_actor {hidden_dim} --hidden_dim_critic {hidden_dim} \
--n_j_options {" ".join(map(str, n_j_options))} --n_m_options {" ".join(map(str, n_m_options))} --op_per_job_options {" ".join(map(str, op_per_job_options))}'
    print(commd, file=file)
    return f"maml+{model_suffix}", commd



def finetuning_maml(model_list, file):
    extra_parameters = f"--hidden_dim_actor {hidden_dim} --hidden_dim_critic {hidden_dim}"
    return finetuning_dan(model_list, file, extra_parameters=extra_parameters)

with open(f"./{SUBEXP}.sh", "w")as f:
    f.write(shell_head)
    model_name, _ = train_maml(f)
    # finetuning_dan(["25x25x10+mix", 
    #                 # "5x5x10+mix"
    #                 ], 
    #                 f)
    finetuning_maml([model_name,], f)



[10, 25, 20, 15] [5, 5, 10, 15] [5, 5, 10, 15]
210


In [56]:
def test_generater(f, test_model, baseline_addition=[]):
    target_data = []
    commd_baseline = []
    if baseline_addition:
        baseline_addition_j = baseline_addition[0]
        baseline_addition_m = baseline_addition[1]
    for n_j in list(range(5, 26, 5)) + baseline_addition_j:
        for n_m in list(range(5, 26, 5)) + baseline_addition_m:
            data_s = f"{n_j}x{n_m}+mix"
            target_data.append(data_s)
            commd_baseline.append(f"python ./test_script/trained_model.py --test_data {data_s} --test_model {n_j}x{n_m}x{'0' if n_j >= 30 else '10'}+mix")
    test_data = " ".join(target_data)
    commd_baseline = "\n".join(commd_baseline)
    print(commd_baseline, file=f)
    for model in test_model:
        if model.startswith("maml"):
            print(f'''python ./test_script/trained_model.py --test_data {test_data} --test_model {model} --hidden_dim_actor {hidden_dim} --hidden_dim_critic {hidden_dim}''', file=f)
        else:
            print(f'''python ./test_script/trained_model.py --test_data {test_data} --test_model {model}''', file=f)

with open("./test1.sh", "w") as f:
    # test_model = ['maml+exp18-9', '25x25x10+mix', '25x20x10+mix', '30x30x10+mix']
    test_model = ['maml+exp18-10']
    baseline_addition=[[40, 30],[20, 10]]
    test_generater(f, test_model, baseline_addition)